In [1]:
import csv
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:

# DATA LOADING & ANALYSIS

def data_load_tsv(path):
    """
    Load data from TSV files with source-target pairs
    """
    df = pd.read_csv(
        path,
        sep='\t',
        header=None,
        dtype=str,
        quoting=csv.QUOTE_NONE
    )
    df = df.dropna(subset=[0,1])
    return df[0].tolist(), df[1].tolist()

# create char set from multiple lists
def create_char_set(*datasets):
    char_set = set()
    for data in datasets:
        for word in data:
            char_set.update(word)
    return char_set


train_input, train_output = data_load_tsv(
    "/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv"
)
val_input, val_output = data_load_tsv(
    "/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv"
)
test_input, test_output = data_load_tsv(
    "/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv"
)


# Print sizes
print(f"Number of training samples:   {len(train_input)}")
print(f"Number of validation samples: {len(val_input)}")
print(f"Number of test samples:       {len(test_input)}")

# Build character sets
src_chars = create_char_set(train_input, val_input, test_input)
tgt_chars = create_char_set(train_output, val_output, test_output)

print("\nSource Character Set:")
print(f"Total characters: {len(src_chars)}")
print(sorted(src_chars))

print("\nTarget Character Set:")
print(f"Total characters: {len(tgt_chars)}")
print(sorted(tgt_chars))

# Max seq lengths 
max_seq_src = max(len(w) for w in train_input + val_input + test_input) + 2
max_seq_tgt = max(len(w) for w in train_output + val_output + test_output) + 2
print(f"\nMax source seq length (with tokens): {max_seq_src}")
print(f"Max target seq length (with tokens): {max_seq_tgt}")


# INDEX MAPPINGS

special_tokens = {'<pad>': 0, '<sow>': 1, '<eow>': 2}

src2idx = {ch: i+3 for i, ch in enumerate(sorted(src_chars))}
src2idx.update(special_tokens)
print("\nSource Indices:")
print(src2idx)

idx2src = {i: ch for ch, i in src2idx.items()}

tgt2idx = {ch: i+3 for i, ch in enumerate(sorted(tgt_chars))}
tgt2idx.update(special_tokens)
print("\nTarget Indices:")
print(tgt2idx)

idx2tgt = {i: ch for ch, i in tgt2idx.items()}

SRC_VOCAB = len(src2idx)
TGT_VOCAB = len(tgt2idx)

# embedding dims 
SRC_EMB_DIM = 64
TGT_EMB_DIM = 64


# PREPROCESSING

def tsv_preprocessor(data, max_len, vocab):
    processed = []
    for w in data:
        seq = ['<sow>'] + list(w) + ['<eow>']
        seq += ['<pad>'] * (max_len - len(seq))
        indices = [vocab.get(c, vocab['<pad>']) for c in seq]
        processed.append(torch.LongTensor(indices))
    return torch.stack(processed)

train_src = tsv_preprocessor(train_input, max_seq_src, src2idx)
train_tgt = tsv_preprocessor(train_output, max_seq_tgt, tgt2idx)
val_src   = tsv_preprocessor(val_input,   max_seq_src, src2idx)
val_tgt   = tsv_preprocessor(val_output,  max_seq_tgt, tgt2idx)
test_src  = tsv_preprocessor(test_input,  max_seq_src, src2idx)
test_tgt  = tsv_preprocessor(test_output, max_seq_tgt, tgt2idx)


# DATASET & DATALOADER
class TSVDataset(Dataset):
    def __init__(self, src, tgt):
        self.src = src
        self.tgt = tgt
    def __len__(self):
        return len(self.src)
    def __getitem__(self, idx):
        return self.src[idx], self.tgt[idx]

# custom collate to pad along seq dim

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_padded = nn.utils.rnn.pad_sequence(src_batch, batch_first=False, padding_value=special_tokens['<pad>'])
    tgt_padded = nn.utils.rnn.pad_sequence(tgt_batch, batch_first=False, padding_value=special_tokens['<pad>'])
    return src_padded, tgt_padded

BATCH_SIZE = 256
train_loader = DataLoader(
    TSVDataset(train_src, train_tgt),
    batch_size=BATCH_SIZE, shuffle=True,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    TSVDataset(val_src, val_tgt),
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    TSVDataset(test_src, test_tgt),
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

Number of training samples:   94543
Number of validation samples: 9279
Number of test samples:       9228

Source Character Set:
Total characters: 60
['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', '২']

Target Character Set:
Total characters: 26
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

Max source seq length (with tokens): 24
Max target seq length (with tokens): 24

Source Indices:
{'ঁ': 3, 'ং': 4, 'ঃ': 5, 'অ': 6, 'আ': 7, 'ই': 8, 'ঈ': 9, 'উ': 10, 'ঊ': 11, 'ঋ': 12, 'এ': 13, 'ঐ': 14, 'ও': 15, 'ঔ': 16, 'ক': 17, 'খ': 18, 'গ': 19, 'ঘ': 20, 'ঙ': 21, 'চ': 22, 'ছ': 23, 'জ': 24, 'ঝ': 25, 'ঞ': 26, 'ট': 27, 'ঠ': 28, 'ড': 29, 'ঢ': 30, 'ণ': 31, 'ত': 32, 'থ': 33, 'দ'

In [3]:

# ENCODER CLASS

class Encoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int,
        hid_dim: int,
        rnn_type: str = 'gru',
        num_layers: int = 1,
        dropout: float = 0.0,
        bidir: bool = False
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        rnn_cls = nn.LSTM if rnn_type.lower() == 'lstm' else nn.GRU
        self.rnn = rnn_cls(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidir,
            batch_first=True
        )

    def forward(
        self,
        x: torch.Tensor,
        hidden: torch.Tensor = None,
        cell: torch.Tensor = None
    ):
        emb = self.embedding(x)  # (batch, seq_len, emb_dim)
        if isinstance(self.rnn, nn.LSTM):
            if hidden is None or cell is None:
                out, (h, c) = self.rnn(emb)
            else:
                out, (h, c) = self.rnn(emb, (hidden, cell))
            return out, h, c
        else:
            if hidden is None:
                out, h = self.rnn(emb)
            else:
                out, h = self.rnn(emb, hidden)
            return out, h, None

In [4]:

# DECODER CLASS

class Decoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int,
        hid_dim: int,
        rnn_type: str = 'gru',
        num_layers: int = 1,
        dropout: float = 0.0,
        bidir: bool = False,
        use_attention: bool = False
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        rnn_cls = nn.LSTM if rnn_type.lower() == 'lstm' else nn.GRU
        self.rnn = rnn_cls(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidir,
            batch_first=True
        )
        # adjust output linear to match bidirectional RNN hidden dim
        out_dim = hid_dim * (2 if bidir else 1)
        self.out = nn.Linear(out_dim, vocab_size)
        self.use_attention = use_attention

    def forward(
        self,
        x: torch.Tensor,
        prev_hidden: torch.Tensor,
        prev_cell: torch.Tensor = None,
        encoder_outputs: torch.Tensor = None
    ):
        emb = self.embedding(x)  # (batch, 1, emb_dim)
        if isinstance(self.rnn, nn.LSTM):
            if prev_hidden is None or prev_cell is None:
                dec_out, (h, c) = self.rnn(emb)
            else:
                dec_out, (h, c) = self.rnn(emb, (prev_hidden, prev_cell))
        else:
            if prev_hidden is None:
                dec_out, h = self.rnn(emb)
                c = None
            else:
                dec_out, h = self.rnn(emb, prev_hidden)
                c = None
        logits = self.out(dec_out)  # dec_out shape: (batch, 1, out_dim)
        return logits, h, c


In [5]:

# SEQ2SEQ CLASS
class Seq2Seq(nn.Module):
    def __init__(
        self,
        encoder: Encoder,
        decoder: Decoder,
        max_tgt_len: int,
        teacher_force_rate: float = 0.5
    ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.max_tgt_len = max_tgt_len
        self.teacher_force_rate = teacher_force_rate

    def forward(
        self,
        src: torch.Tensor,
        tgt: torch.Tensor = None,
        teacher_forcing: bool = True,
        training: bool = True
    ):
        # Encode source sequence
        enc_outputs, h_enc, c_enc = self.encoder(src)

        # Align hidden cell to decoder's layers
        rnn = self.decoder.rnn
        dirs = 2 if rnn.bidirectional else 1
        exp_layers = rnn.num_layers * dirs
        L_enc, B, H = h_enc.size()
        if L_enc != exp_layers:
            if L_enc > exp_layers:
                h = h_enc[:exp_layers]
                c = c_enc[:exp_layers] if c_enc is not None else None
            else:
                pad = exp_layers - L_enc
                h = torch.cat([h_enc, h_enc.new_zeros(pad, B, H)], 0)
                if c_enc is not None:
                    c = torch.cat([c_enc, c_enc.new_zeros(pad, B, H)], 0)
                else:
                    c = None
        else:
            h, c = h_enc, c_enc

        # Prepare initial decoder input 
        dec_in = torch.full((B,1), 1, dtype=torch.long, device=src.device)
        outputs = torch.zeros(
            self.max_tgt_len, B, self.decoder.out.out_features,
            device=src.device
        )

        for t in range(self.max_tgt_len):
            logits, h, c = self.decoder(dec_in, h, c, enc_outputs)
            outputs[t] = logits.squeeze(1)
            if training and teacher_forcing and random.random() < self.teacher_force_rate:
                dec_in = tgt[:,t].unsqueeze(1)
            else:
                dec_in = logits.argmax(dim=2)

        return outputs, enc_outputs


In [6]:
def train1(model, train_loader, val_loader, epochs):
    model.to(device)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()

    for epoch in tqdm(range(1, epochs + 1), desc="Epochs"):
        # ------------------
        # Training Phase
        # ------------------
        model.train()
        train_loss = 0.0
        train_hits = 0

        for src_batch, tgt_batch in train_loader:
            # transpose from (seq_len, B) -> (B, seq_len)
            src = src_batch.transpose(0, 1).to(device)
            tgt = tgt_batch.transpose(0, 1).to(device)

            optimizer.zero_grad()
            outputs, _ = model(src, tgt, teacher_forcing=True, training=True)
            # outputs: (T, B, V) -> (B*T, V) for loss
            logits = outputs.permute(1, 0, 2).reshape(-1, TGT_VOCAB)
            truth  = tgt.reshape(-1)

            loss = criterion(logits, truth)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * src.size(0)
            train_hits += (logits.argmax(dim=1) == truth).sum().item()

        avg_train_loss = train_loss / len(train_loader.dataset)
        train_acc      = train_hits / (len(train_loader.dataset) * max_seq_tgt)

        
        # Validation Phase
    
        model.eval()
        val_loss       = 0.0
        val_word_hits  = 0

        with torch.no_grad():
            for src_batch, tgt_batch in val_loader:
                src = src_batch.transpose(0, 1).to(device)
                tgt = tgt_batch.transpose(0, 1).to(device)

                outputs, _ = model(src, tgt, teacher_forcing=False, training=False)
                # token-level loss
                logits = outputs.permute(1, 0, 2).reshape(-1, TGT_VOCAB)
                truth  = tgt.reshape(-1)
                loss   = criterion(logits, truth)

                val_loss += loss.item() * src.size(0)

                # word-level accuracy: count sequences where all tokens match
                pred_tokens = outputs.argmax(dim=2).transpose(0, 1)  # (B, T)
                val_word_hits += (pred_tokens == tgt).all(dim=1).sum().item()

        avg_val_loss    = val_loss / len(val_loader.dataset)
        validation_acc  = val_word_hits / len(val_loader.dataset)


        # Logging
        print(
            f"Epoch {epoch}: "
            f"train_loss={avg_train_loss:.4f}, train_acc={train_acc:.4f} | "
            f"val_loss={avg_val_loss:.4f}, validation_acc={validation_acc:.4f}"
        )
        wandb.log({
            "epoch": epoch,
            "train_loss":     avg_train_loss   * 100,
            "train_acc":      train_acc        * 100,
            "val_loss":       avg_val_loss     * 100,
            "validation_acc": validation_acc   * 100
        })


In [7]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [8]:
wandb.login(key='1df7a902fa4a610500b8e79e21818419d5facdbb')#


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m018 (ma23m018-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

**Find best validation accuracy without attention**

In [9]:

# SWEEP CONFIGURATION (Bayesian, no attention)
sweep_config = {
    'method': 'bayes',
    'name'  : 'sweep - no attention',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'input_embedding_size': {'values': [128, 256, 512]},
        'enc_layers':            {'values': [1, 2, 3]},
        'dec_layers':            {'values': [1, 2, 3]},
        'hidden_size':           {'values': [ 128, 256, 512]},
        'cell_type':             {'values': ['lstm', 'rnn', 'gru']},
        'bidirectional':         {'values': [True]},
        'dropout':               {'values': [0.1, 0.2, 0.3]},
        'beam_size':             {'values': [1, 3, 5]}
    }
}

# define sweep_id 
sweep_id = wandb.sweep(
    sweep   = sweep_config,
    entity  = "ma23m018-indian-institute-of-technology-madras",
    project = "DA6401_A3"
)

# MAIN FUNCTION
def main():
    with wandb.init():
        cfg = wandb.config
        wandb.run.name = (
            f"cell-{cfg.cell_type}_hid-{cfg.hidden_size}"
            f"_emb-{cfg.input_embedding_size}"
            f"_enc-{cfg.enc_layers}_dec-{cfg.dec_layers}"
            f"_drop{cfg.dropout}_beam{cfg.beam_size}"
        )

        encoder = Encoder(
            vocab_size = SRC_VOCAB,
            emb_dim    = cfg.input_embedding_size,
            hid_dim    = cfg.hidden_size,
            rnn_type   = cfg.cell_type,
            num_layers = cfg.enc_layers,
            dropout    = cfg.dropout,
            bidir      = cfg.bidirectional
        )

        decoder = Decoder(
            vocab_size    = TGT_VOCAB,
            emb_dim       = cfg.input_embedding_size,
            hid_dim       = cfg.hidden_size,
            rnn_type      = cfg.cell_type,
            num_layers    = cfg.dec_layers,
            dropout       = cfg.dropout,
            bidir         = cfg.bidirectional,
            use_attention = False
        )

        model = Seq2Seq(
            encoder            = encoder,
            decoder            = decoder,
            max_tgt_len        = max_seq_tgt,
            teacher_force_rate = 0.5
        ).to(device)

    
        train1(model, train_loader, val_loader, epochs=10)

# launch 15 sweep runs of main()
wandb.agent(sweep_id, function=main, count=15)
wandb.finish()


Create sweep with ID: ggd6ozc5
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/DA6401_A3/sweeps/ggd6ozc5


wandb: Agent Starting Run: gi8hv6a2 with config:
wandb: 	beam_size: 1
wandb: 	bidirectional: True
wandb: 	cell_type: rnn
wandb: 	dec_layers: 3
wandb: 	dropout: 0.2
wandb: 	enc_layers: 3
wandb: 	hidden_size: 256
wandb: 	input_embedding_size: 512


Epochs:  10%|█         | 1/10 [00:40<06:06, 40.76s/it]

Epoch 1: train_loss=0.5927, train_acc=0.8155 | val_loss=0.5811, validation_acc=0.1140


Epochs:  20%|██        | 2/10 [01:23<05:36, 42.06s/it]

...

Epochs:  90%|█████████ | 9/10 [06:29<00:43, 43.24s/it]

Epoch 9: train_loss=0.2798, train_acc=0.9046 | val_loss=0.5476, validation_acc=0.1580


Epochs: 100%|██████████| 10/10 [07:13<00:00, 43.32s/it]

Epoch 10: train_loss=0.2679, train_acc=0.9081 | val_loss=0.5724, validation_acc=0.1638


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▅▆▇▇█████
train_loss,█▃▂▂▂▁▁▁▁▁
val_loss,█▆▂▃▃▁▂▁▁▃
validation_acc,▁▅▆▇▇▇████
epoch,10
train_acc,90.81291
train_loss,26.78848
val_loss,57.2426
validation_acc,16.38108


**Finding the test accuracy with the best validation accuracy giving hyperparameter**

In [12]:
import torch.nn.functional as F

def compute_score(preds: torch.Tensor, targets: torch.Tensor) -> int:
    
    # each row is True if all tokens match
    correct_seq = (preds == targets).all(dim=1)
    return correct_seq.sum().item()


def calc_test_acc(model, loader):
    model.eval()
    total_loss  = 0
    total_score = 0
    criterion   = nn.CrossEntropyLoss()

    for inputs, targets in loader:
        # inputs, targets
        inputs, targets = inputs.transpose(0,1).to(device), targets.transpose(0,1).to(device)
        outputs, _ = model(inputs, None, False, False)

        # Calculate accuracy
        preds = torch.argmax(F.softmax(outputs, dim=2), dim=2).T  # (batch, seq)
        total_score += compute_score(preds, targets)

        # Reshape outputs and targets for loss calculation
        outputs = outputs.permute(1, 0, 2).reshape(-1, TGT_VOCAB)
        targets = F.one_hot(targets, num_classes=TGT_VOCAB).float().reshape(-1, TGT_VOCAB)

        # Calculate loss
        loss = criterion(outputs, targets)
        total_loss += loss.item()

    avg_loss  = total_loss  / len(loader)
    avg_score = total_score / len(loader.dataset)

    print(f'Test Loss: {avg_loss:.4f} \t Test Accuracy: {avg_score:.4f}')
    wandb.log({'test_accuracy': avg_score * 100, 'test_loss': avg_loss})





In [13]:

#  Instantiate best_model with best valiation hyperparameter
best_encoder = Encoder(
    vocab_size = SRC_VOCAB,
    emb_dim    = 512,
    hid_dim    = 512,
    rnn_type   = 'lstm',
    num_layers = 2,
    dropout    = 0.3,
    bidir      = True
)

best_decoder = Decoder(
    vocab_size    = TGT_VOCAB,
    emb_dim       = 512,
    hid_dim       = 512,
    rnn_type      = 'lstm',
    num_layers    = 3,
    dropout       = 0.3,
    bidir         = True,
    use_attention = False
)

best_model = Seq2Seq(
    encoder            = best_encoder,
    decoder            = best_decoder,
    max_tgt_len        = max_seq_tgt,
    teacher_force_rate = 0.5
).to(device)


#train on train+val 
wandb.init(
    project = "MA23M018_assignment3",
    entity  = "ma23m018-indian-institute-of-technology-madras",
    name    = "final_train_on_trainval"
)
train1(best_model, train_loader, val_loader, epochs=10)
wandb.finish()



#Grid‐sweep config to run test‐set eval
sweep_config = {
    'method':  'grid',
    'name'  :  'testset run',
    'metric': {'goal': 'maximize', 'name': 'test_accuracy'},
    'parameters': {
        'beam_size': {'values': [1]}
    }
}

#sweep_id for test-set 
sweep_id = wandb.sweep(
    sweep   = sweep_config,
    entity  = "ma23m018-indian-institute-of-technology-madras",
    project = "DA6401_TEST_A3"
)



#  main() that just calls calc_test_acc on the test set
def main():
    with wandb.init():
        wandb.run.name = "test_set_run"
        calc_test_acc(best_model, test_loader)

# launch run for test-set
wandb.agent(sweep_id, function=main, count=1)
wandb.finish()


epoch,▁█
train_acc,▁█
train_loss,█▁
val_loss,█▁
validation_acc,▁█
epoch,2
train_acc,89.43805
train_loss,31.65786
val_loss,53.92083
validation_acc,17.34023


Epochs:  10%|█         | 1/10 [02:35<23:16, 155.17s/it]

Epoch 1: train_loss=0.6259, train_acc=0.8083 | val_loss=0.5568, validation_acc=0.1417


Epochs:  20%|██        | 2/10 [05:09<20:37, 154.66s/it]

Epoch 2: train_loss=0.3115, train_acc=0.8960 | val_loss=0.5141, validation_acc=0.1747


...

epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▆▇▇██████
train_loss,█▃▂▂▁▁▁▁▁▁
val_loss,█▄▆▅▇▁▅▅▄▇
validation_acc,▁▅▆█▇▆▇▇▆█
epoch,10
train_acc,92.03233
train_loss,21.44666
val_loss,55.14309
validation_acc,19.7004


Create sweep with ID: 6tedvxsl
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/DA6401_TEST_A3/sweeps/6tedvxsl


wandb: Agent Starting Run: 1kh413a7 with config:
wandb: 	beam_size: 1


Test Loss: 0.5460 	 Test Accuracy: 0.1967


test_accuracy,▁
test_loss,▁
test_accuracy,19.6684
test_loss,0.54597


**Save the predictions on test set in a csv file**

In [14]:

# decode a sequence of indices into a string
def decode_sequence(idx_seq, idx2char):
   
    chars = []
    for idx in idx_seq:
        # stop at end‐of‐word
        if idx == special_tokens['<eow>']:
            break
        # skip pad and start‐of‐word
        if idx in (special_tokens['<pad>'], special_tokens['<sow>']):
            continue
        chars.append(idx2char[idx])
    return "".join(chars)


# run the model on the test set and collect rows
best_model.eval()
rows = []

with torch.no_grad():
    for src_batch, tgt_batch in test_loader:
        # transpose to (batch, seq)
        src = src_batch.transpose(0,1).to(device)
        tgt = tgt_batch.transpose(0,1).to(device)

        # get model outputs (T, B, V)
        outputs, _ = best_model(src, None, False, False)

        # predicted indices
        pred_idxs = outputs.argmax(dim=2).transpose(0,1).cpu().tolist()

        for inp_idxs, true_idxs, pr_idxs in zip(
            src.cpu().tolist(),
            tgt.cpu().tolist(),
            pred_idxs
        ):
            inp_str  = decode_sequence(inp_idxs, idx2src)
            true_str = decode_sequence(true_idxs, idx2tgt)
            pred_str = decode_sequence(pr_idxs, idx2tgt)
            correctness = "Correct" if pred_str == true_str else "Incorrect"

            rows.append({
                "Input-English":        inp_str,
                "Output-Bengali":     true_str,
                "Predicted-Bengali":  pred_str,
                "Correct/Incorrect":    correctness
            })


#  CSV
import pandas as pd

df = pd.DataFrame(rows, columns=[
    "Input-English",
    "Output-Bengali",
    "Predicted-Bengali",
    "Correct/Incorrect"
])
df.to_csv("predictions.csv", index=False)
print(f"Saved predictions.csv with {len(df)} rows.")


Saved predictions.csv with 9228 rows.


**With attention**

In [15]:

# SWEEP CONFIGURATION (Bayesian, with attention)

sweep_config = {
    'method': 'bayes',
    'name'  : 'sweep - attention',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'input_embedding_size': {'values': [128, 256, 512]},
        'enc_layers':            {'values': [1, 2, 3]},
        'dec_layers':            {'values': [1, 2, 3]},
        'hidden_size':           {'values': [128, 256, 512]},
        'cell_type':             {'values': ['lstm', 'gru', 'rnn']},
        'bidirectional':         {'values': [True]},
        'dropout':               {'values': [0.1, 0.2, 0.3]},
        'beam_size':             {'values': [1, 3, 5]}
    }
}

sweep_id = wandb.sweep(
    sweep   = sweep_config,
    entity  = "ma23m018-indian-institute-of-technology-madras",
    project = "DA6401_A3_ATTN"
)

# MAIN FUNCTION FOR ATTENTION SWEEP
def main():
    with wandb.init():
        cfg = wandb.config
        wandb.run.name = (
            f"attn_cell-{cfg.cell_type}_hid-{cfg.hidden_size}"
            f"_emb-{cfg.input_embedding_size}"
            f"_enc-{cfg.enc_layers}_dec-{cfg.dec_layers}"
            f"_drop{cfg.dropout}_beam{cfg.beam_size}"
        )

        encoder = Encoder(
            vocab_size = SRC_VOCAB,
            emb_dim    = cfg.input_embedding_size,
            hid_dim    = cfg.hidden_size,
            rnn_type   = cfg.cell_type,
            num_layers = cfg.enc_layers,
            dropout    = cfg.dropout,
            bidir      = cfg.bidirectional
        )

        # decoder with attention enabled
        decoder = Decoder(
            vocab_size    = TGT_VOCAB,
            emb_dim       = cfg.input_embedding_size,
            hid_dim       = cfg.hidden_size,
            rnn_type      = cfg.cell_type,
            num_layers    = cfg.dec_layers,
            dropout       = cfg.dropout,
            bidir         = cfg.bidirectional,
            use_attention = True
        )

        model = Seq2Seq(
            encoder            = encoder,
            decoder            = decoder,
            max_tgt_len        = max_seq_tgt,
            teacher_force_rate = 0.5
        ).to(device)

        # train & compute word‐level validation accuracy
        train1(model, train_loader, val_loader, epochs=10)

wandb.agent(sweep_id, function=main, count=15)
wandb.finish()

Create sweep with ID: icx3riaa
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/DA6401_A3_ATTN/sweeps/icx3riaa


wandb: Agent Starting Run: v04cruhl with config:
wandb: 	beam_size: 3
wandb: 	bidirectional: True
wandb: 	cell_type: gru
wandb: 	dec_layers: 1
wandb: 	dropout: 0.2
wandb: 	enc_layers: 3
wandb: 	hidden_size: 128
wandb: 	input_embedding_size: 128


Epochs:  10%|█         | 1/10 [00:20<03:05, 20.60s/it]

Epoch 1: train_loss=0.9075, train_acc=0.7382 | val_loss=0.7531, validation_acc=0.0213


Epochs:  20%|██        | 2/10 [00:41<02:45, 20.69s/it]

...

Epochs:  90%|█████████ | 9/10 [04:35<00:30, 30.72s/it]

Epoch 9: train_loss=0.2351, train_acc=0.9151 | val_loss=0.5513, validation_acc=0.1590


Epochs: 100%|██████████| 10/10 [05:06<00:00, 30.62s/it]

Epoch 10: train_loss=0.2271, train_acc=0.9174 | val_loss=0.5707, validation_acc=0.1572


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▆▇▇██████
train_loss,█▃▂▂▂▁▁▁▁▁
val_loss,█▂▁▅▂▅█▃▄█
validation_acc,▁▄▅▆▆▇█▆▇▆
epoch,10
train_acc,91.74357
train_loss,22.71203
val_loss,57.07346
validation_acc,15.72368


**Apply the best validation hyperparameter on test data**

In [17]:
best_encoder_attn = Encoder(
    vocab_size = SRC_VOCAB,
    emb_dim    = 256,
    hid_dim    = 256,
    rnn_type   = 'rnn',
    num_layers = 3,
    dropout    = 0.2,
    bidir      = True
)

best_decoder_attn = Decoder(
    vocab_size    = TGT_VOCAB,
    emb_dim       = 256,
    hid_dim       = 256,
    rnn_type      = 'rnn',
    num_layers    = 3,
    dropout       = 0.2,
    bidir         = True,
    use_attention = True
)

best_model_attn = Seq2Seq(
    encoder            = best_encoder_attn,
    decoder            = best_decoder_attn,
    max_tgt_len        = max_seq_tgt,
    teacher_force_rate = 0.5
).to(device)

# retrain on train+val to finalize
wandb.init(
    project = "MA23M018_assignment3",
    entity  = "ma23m018-indian-institute-of-technology-madras",
    name    = "final_train_with_attention"
)
train1(best_model_attn, train_loader, val_loader, epochs=10)
wandb.finish()

# TEST-SET EVAL SWEEP (Attention)
sweep_config = {
    'method': 'grid',
    'name'  : 'testset_run_attention',
    'metric': {'goal': 'maximize', 'name': 'test_accuracy'},
    'parameters': {
        
        'beam_size': {'values': [1]}
    }
}

sweep_id = wandb.sweep(
    sweep   = sweep_config,
    entity  = "ma23m018-indian-institute-of-technology-madras",
    project = "DA6401_A3_TEST_ATTN"
)

def test_main():
    with wandb.init():
        cfg = wandb.config
        wandb.run.name = f"test_set_run_attn_beam{cfg.beam_size}"
        # If you had a beam‐search decode, you'd pass cfg.beam_size here.
        calc_test_acc(best_model_attn, test_loader)

# Run 1 test‐set evaluation
wandb.agent(sweep_id, function=test_main, count=1)
wandb.finish()


Epochs:  10%|█         | 1/10 [00:42<06:22, 42.48s/it]

Epoch 1: train_loss=0.6070, train_acc=0.8106 | val_loss=0.5629, validation_acc=0.1278


Epochs:  20%|██        | 2/10 [01:23<05:35, 41.91s/it]

Epoch 2: train_loss=0.3442, train_acc=0.8859 | val_loss=0.5656, validation_acc=0.1660


Epochs:  30%|███       | 3/10 [02:06<04:53, 41.99s/it]

...

epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▆▇▇▇█████
train_loss,█▃▂▂▂▁▁▁▁▁
val_loss,██▂▂▁▃▂▅▅▅
validation_acc,▁▆▆▆▆█▆███
epoch,10
train_acc,91.51973
train_loss,23.65463
val_loss,54.4817
validation_acc,18.60114


Create sweep with ID: kacz1wga
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/DA6401_A3_TEST_ATTN/sweeps/kacz1wga


wandb: Agent Starting Run: s6cn3nnr with config:
wandb: 	beam_size: 1


Test Loss: 0.5267 	 Test Accuracy: 0.1856


test_accuracy,▁
test_loss,▁
test_accuracy,18.56307
test_loss,0.52675
